In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/data_for_upset.tsv',sep='\t')

# Clean up so only have cols of the 6 groups
df = df[[
    'PRAD:CF|All_Top 100', 
    'nn_jg_2020-03-20_top1kfreq:PRAD',
    'AKLIMATE_PRAD_reduced_model_1500_feature_set', 
    'gnosis:PRAD:9', 
    'fbed_combined',
    'rfe_combined'
]]
df

In [ ]:
df.columns

# How many overlapping features within a group?

ex. are there overlapping features within Gnosis selected features alone?

In [ ]:
import plotly.graph_objects as go

def coords(gene_dict, subset_size):
    x = []
    y = []
    for pair in gene_dict[:subset_size]:
        x.append(pair[0])
        y.append(pair[1])
    return x, y


In [ ]:
# what does NA mean in N:METH:cg05688651:NA:NA:Island ??
cols=df.columns
for i in range(0, len(cols)):
    gene_dict ={}
    title = cols[i]
    for ft in df[cols[i]].dropna():
        gene = ft.split(':')[3]
        if gene not in gene_dict:
            gene_dict[gene]=1
        else:
            gene_dict[gene]+=1
    gene_dict = sorted(gene_dict.items(), key=lambda item: item[1],reverse=True)

    # Plot
    x, y = coords(gene_dict, 10)
    layout = go.Layout(title = "Within Group: {}".format(title), yaxis = dict(title='Frequency'), xaxis = dict(title='Most abundant features', tickangle=45) )
    fig = go.Figure([go.Bar(x=x, y=y)], layout)
    fig.show()

In [ ]:
# quick tangent on looking at the NA features ( NA mean in N:METH:cg05688651:NA:NA:Island ?? )
gene_dict = {}
cols=df.columns
for i in range(0, len(cols)):
    title = cols[i]
    for ft in df[cols[i]].dropna():
        gene = ft.split(':')[3]
        if gene == 'NA':
            if gene not in gene_dict:
                gene_dict[ft]=[title]
            else:
                gene_dict[ft].append(title)
sorted_gene_dict = sorted(gene_dict.items(), key=lambda item: item[1],reverse=True)
for a in gene_dict.keys():
    print(a)

In [ ]:
df

# How many overlapping features within and across groups?

ex. are there overlapping features when pool all 6 groups together?

In [ ]:
cols=df.columns
gene_dict ={}

for i in range(0, len(cols)):
    for ft in df[cols[i]].dropna():
        gene = ft.split(':')[3]
        if gene not in gene_dict:
            gene_dict[gene]=1
        else:
            gene_dict[gene]+=1
gene_dict = sorted(gene_dict.items(), key=lambda item: item[1],reverse=True)

In [ ]:
# Plot - all but messy because lots of NA gene symbols
x, y = coords(gene_dict, 50)
layout = go.Layout(title = "Feature selection methods pick the same genes (regardless of gene variant type)", yaxis = dict(title='Frequency'), xaxis = dict(title='Most abundant features', tickangle=45) )
fig = go.Figure(
    [go.Bar(
        x=x, 
        y=y,
        text=y,
        textposition='outside'
    )], 
    layout
)
fig.show()

# What are these features?

for a selected gene symbol bin, what is the composition of features?

In [ ]:
import plotly.graph_objects as go


In [ ]:
####
selected_gene = 'HSPB1'
####

In [ ]:
# Reformat data
cols=df.columns
gene_dict ={}

for i in range(0, len(cols)):
    title = cols[i]
    for ft in df[cols[i]].dropna():
        gene = ft.split(':')[3]
        if gene == selected_gene:
            if ft not in gene_dict:
                gene_dict[ft]=[title]
            else:
                gene_dict[ft].append(title)

In [ ]:
# 1. Which platforms contributed? - pie chart
results_dict = {}
for ft, groups in gene_dict.items():
    data_type = ft.split(':')[1]
    if data_type not in results_dict:
        results_dict[data_type]=len(groups)
    else:
        results_dict[data_type]+=len(groups)


labels = []
values = []
for k, v in results_dict.items():
    labels.append(k)
    values.append(v)
fig = go.Figure(data=[go.Pie(labels=labels, values=values,textinfo='label+percent')])
fig.update(layout_showlegend=False)
fig.show()



# 2. Which groups contributed?
results_dict = {}
for ft, groups in gene_dict.items():
    for g in groups:
        if g not in results_dict:
            results_dict[g]=1
        else:
            results_dict[g]+=1

            
labels = []
values = []
for k, v in results_dict.items():
    labels.append(k)
    values.append(v)
fig = go.Figure(data=[go.Pie(labels=labels, values=values,textinfo='label+percent')])
fig.update(layout_showlegend=False)
fig.show()



# 3. How often does a specific gene variant occur?
x = []
y = []
for ft, groups in gene_dict.items():
    x.append(ft)
    y.append(len(groups))
    
layout = go.Layout(
    yaxis = dict(title='Frequency'), 
    xaxis = dict(
        tickangle=45,
        categoryorder='array',
        categoryarray=[x for _, x in sorted(zip(y, x), reverse=True)]
    ) 
)

fig = go.Figure(
    [go.Bar(
        x=x, 
        y=y,
        text=y,
        textposition='inside'
    )], 
    layout
)
fig.show()

In [ ]:
gene_dict.keys()

# print this out for upset plot

In [ ]:
original_df = df
original_df

In [ ]:
# reread in file to double check things have not changed
df = pd.read_csv('../data/data_for_upset.tsv',sep='\t')

# Clean up so only have cols of the 6 groups
df = df[[
    'PRAD:CF|All_Top 100', 
    'nn_jg_2020-03-20_top1kfreq:PRAD',
    'AKLIMATE_PRAD_reduced_model_1500_feature_set', 
    'gnosis:PRAD:9', 
    'fbed_combined',
    'rfe_combined'
]]

In [ ]:
# another quick tangent: create output df for upset plot
def get_output(input_df):
    output_df = df # prep for later
    cols = input_df.columns
    

    for i in range(0, len(cols)):
        new_col = []
        header = cols[i]
        for ft in input_df[header]:
            if type(ft) == float and str(ft) == 'nan':
                new_col.append(ft)
            else:
                gene = ft.split(':')[3]
                if gene == 'NA':
                    gene = 'NA-feature'
                new_col.append(gene)
        if i == 0:
            col1 = new_col
            col1_header = header
        elif i == 1:
            cleaned_df = pd.DataFrame(list(zip(col1, new_col)),columns=[col1_header, header])
        else:
            cleaned_df[header]= new_col
    return cleaned_df



In [ ]:
cleaned_df = df
cleaned_df = get_output(cleaned_df)
cleaned_df

In [ ]:
cleaned_df.to_csv('../data/GENE_only_data_for_upset.tsv',sep='\t',index=False)